In [1]:
#pip install selenium pandas beautifulsoup4
#only need to install if not already installed if unsure run anyway and the machine will tell you

In [2]:
import pandas as pd - #as always 
import time # waits before loading the next webpage (be nice to the servers and the machine)
import re #pattern-matching in this case we are looking for works that commonly become before a skill we want to track
from collections import Counter # exactly what is sounds like counts instances
from bs4 import BeautifulSoup # parses html
from selenium import webdriver # IMPORTANT CAN VARY MAKE SURE DRIVER AND VERSION ARE THE SAME
from selenium.webdriver.chrome.options import Options #virtual browser you can control with Python

In [3]:
# Load the file that contains LinkedIn URLs
df = pd.read_csv('C:/Users/holle/OneDrive/Documents/Data Sets (Public)/LinkedIn Pull of Analytics Jobs.csv')  # Make sure your file has a column called 'link'
#df.head()

,id,title,company,location,link,source,date_posted,work_type,employment_type,description
0,1,Data Analyst,Meta,"New York, NY",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,4/14/2025,NaN,NaN,The Social Measurement team is a growing team ...
1,2,Data Analyst,Meta,"San Francisco, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,4/14/2025,NaN,NaN,The Social Measurement team is a growing team ...
2,3,Data Analyst,Meta,"Los Angeles, CA",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,4/14/2025,NaN,NaN,The Social Measurement team is a growing team ...
3,4,Data Analyst,Meta,"Washington, DC",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,4/14/2025,NaN,NaN,The Social Measurement team is a growing team ...
4,5,Data Analyst II,Pinterest,"Chicago, IL",https://www.linkedin.com/jobs/view/data-analys...,LinkedIn,4/16/2025,NaN,NaN,About Pinterest\n\nMillions of people around t...


In [4]:
# Set up Chrome in headless mode so it runs in the background (personal preference)
#I only ever have to do this for web scraping
options = Options()
options.add_argument('--headless')  # Comment this out if you want to watch the browser
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)  # Ensure chromedriver is in your system PATH

In [5]:
# Define patterns to detect skill phrases
#can be a living list update as needed
skill_patterns = [
    r'experience with ([\w\s\+\#\-\/]+)',
    r'proficient in ([\w\s\+\#\-\/]+)',
    r'knowledge of ([\w\s\+\#\-\/]+)',
    r'familiar with ([\w\s\+\#\-\/]+)',
    r'skilled in ([\w\s\+\#\-\/]+)',
    r'ability to use ([\w\s\+\#\-\/]+)',
    r'expertise in ([\w\s\+\#\-\/]+)',
]

# Extract job description text from LinkedIn page
#define funcition that get the job description
def get_job_description(url):
    try:
        driver.get(url)
        time.sleep(4)  # wait for the page to load
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        desc_section = soup.find('div', {'class': re.compile('description|show-more-less-html')})
        return desc_section.get_text(separator=' ', strip=True) if desc_section else 'N/A'
    except Exception as e:
        return f'Error: {str(e)}'

# Extract skill phrases from the job description
def extract_skills(text):
    text = str(text).lower()
    skills = []
    for pattern in skill_patterns:
        matches = re.findall(pattern, text)
        cleaned = [m.strip().title() for m in matches if 1 <= len(m.split()) <= 5]
        skills.extend(cleaned)
    return skills

In [6]:
#here we go loop de loop
# this loop is doing the scraping it's going to run until all data in the file (618 links) have run, some will not return anything since the links may be dead the further out this is from posting. 
descriptions = []
skills_by_posting = []

for link in df['link']:
    print(f"Processing: {link}")
    jd = get_job_description(link)
    descriptions.append(jd)
    skills = extract_skills(jd)
    skills_by_posting.append(skills)

Processing: https://www.linkedin.com/jobs/view/data-analyst-at-meta-4186238974
Processing: https://www.linkedin.com/jobs/view/data-analyst-at-meta-4186241553
Processing: https://www.linkedin.com/jobs/view/data-analyst-at-meta-4186236994
Processing: https://www.linkedin.com/jobs/view/data-analyst-at-meta-4186237989
Processing: https://www.linkedin.com/jobs/view/data-analyst-ii-at-pinterest-4193349988
Processing: https://www.linkedin.com/jobs/view/data-analyst-at-fanduel-4206047782
Processing: https://www.linkedin.com/jobs/view/data-analyst-production-finance-operations-innovation-at-netflix-4205626465
Processing: https://www.linkedin.com/jobs/view/data-analyst-marketing-at-fanduel-4138322262
Processing: https://www.linkedin.com/jobs/view/data-analyst-at-sbh-fashion-4168179268
Processing: https://www.linkedin.com/jobs/view/data-analyst-ii-at-pinterest-4193356222
Processing: https://www.linkedin.com/jobs/view/data-analyst-ii-at-pinterest-4193351793
Processing: https://www.linkedin.com/job

In [7]:
# Add new columns to the original DataFrame
df['Job_Description'] = descriptions
df['Extracted_Skills'] = skills_by_posting

# Count total skill mentions across all job descriptions
all_skills = [skill for sublist in skills_by_posting for skill in sublist]
top_skills = Counter(all_skills).most_common(50)
skills_df = pd.DataFrame(top_skills, columns=['Skill', 'Count'])

# Save results 
#be sure to specify what the final files should be called and where they should be saved
df.to_csv('C:/Users/holle/OneDrive/Documents/Data Sets (Public)/JD_Added.csv', index=False)
skills_df.to_csv('C:/Users/holle/OneDrive/Documents/Data Sets (Public)/top_skills_summary.csv', index=False)

In [9]:
#shut down the driver and let yourself know it's done!
driver.quit()
print("It's the end of the script as we know it!")

It's the end of the script as we know it!
